# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.


# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning.

In [ ]:
import mlflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import os

import tensorflow as tf
from keras.layers import Dense, LSTM

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score

: 

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Download latest version
path = kagglehub.dataset_download("muhamedumarjamil/crypto-and-gold-prices-dataset-20152025")

print("Path to dataset files:", path)

In [ ]:
 = pd.read_csv('/kaggle/input/df/crypto-and-gold-prices-dataset-20152025/Crypto Data Since 2015.csv')
df.head()

In [ ]:
# Date ke datetime & set index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [ ]:
# Visualisasi Tren Harga
plt.figure(figsize=(14, 7))
symbols_to_plot = ["Bitcoin (USD)", "Ethereum (USD)", "Gold (USD per oz)"]
for symbol in symbols_to_plot:
    plt.plot(
        df.index,
        df[symbol],
        label=symbol
    )
plt.title("Tren Harga Close Bitcoin, Etherum, & Emas (2015–2025)")
plt.xlabel("Tahun")
plt.ylabel("Harga (USD)")
plt.legend()
plt.show()

In [ ]:
# Visualisasi Tren Harga
plt.figure(figsize=(14, 7))
symbols_to_plot = ["Cardano (ADA)", "Binance Coin (BNB)", "Ripple (XRP)", "Dogecoin (DOGE)", "Solana (SOL)"]
for symbol in symbols_to_plot:
    plt.plot(
        df.index,
        df[symbol],
        label=symbol
    )
plt.title("Tren Harga Close Kripto Lainnya (2015–2025)")
plt.xlabel("Tahun")
plt.ylabel("Harga (USD)")
plt.legend()
plt.show()

In [ ]:
# Calculate daily returns
daily_returns = df.pct_change().dropna()

# Loop to display the daily returns plot for each crypto
for column in daily_returns.columns:
    plt.figure(figsize=(12, 5))
    plt.plot(daily_returns.index, daily_returns[column], label=f"{column}", color="tab:red")

    plt.title(f"Daily Returns - {column}", fontsize=14)
    plt.xlabel("Tahun")
    plt.ylabel("Return Harian")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()

In [ ]:
correlation_matrix = daily_returns.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Korelasi Daily Returns Antar Kripto dan Emas")
plt.show()

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning.

Jika Anda menggunakan data teks, data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah. Khususnya ketika kami menggunakan data tidak terstruktur.

In [ ]:
# Melihat missing value
df.isnull().sum()

In [ ]:
# Melihat data duplikat
print("Jumlah data duplikat:", df.duplicated().sum())

In [ ]:
df.describe()

In [ ]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

In [ ]:
selected_columns = ["Bitcoin (USD)", "Ethereum (USD)", "Gold (USD per oz)", "Cardano (ADA)", "Binance Coin (BNB)", "Ripple (XRP)", "Dogecoin (DOGE)", "Solana (SOL)"]
N_FEATURES = len(selected_columns)
print("jumlah fitur yang digunakan:", N_FEATURES)

In [ ]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
# Split data into training and validation sets
train_size = int(len(data) * 0.8)
x_train, x_valid = data[0:train_size], data[train_size:]

In [ ]:
BATCH_SIZE = 32
N_PAST = 30
N_FUTURE = 30
SHIFT = 1
# Kode untuk membuat windowed datasets
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

# Modelling

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(N_PAST, N_FEATURES)),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(N_FUTURE * N_FEATURES), # Change units to N_FUTURE * N_FEATURES
    tf.keras.layers.Reshape((N_FUTURE, N_FEATURES))
])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('mae') < 0.055 and logs.get('val_mae') < 0.055):
                self.model.stop_training = True

callbacks = myCallback()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='mae',
                  optimizer= optimizer,
                  metrics=["mae"])

In [ ]:
model.fit(train_set,
          validation_data=(valid_set),
          epochs=100,
          callbacks=callbacks,
          verbose=1
    )

In [ ]:
train_pred = model.predict(train_set)
train_pred[0][0]

In [ ]:
crypto_names = ["BTC", "ETH", "GOLD", "ADA", "BNB", "XRP", "DOGE", "SOL"]
train_pred = model.predict(train_set)
predictions = train_pred[0]   # shape (N_FUTURE, n_crypto)

# Data tanggal
last_date = pd.to_datetime("2025-07-24")   # data terakhir
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1),
                             periods=len(predictions), freq="D")

# Visualisasi Subplot
n_crypto = len(crypto_names)
n_cols = 2
n_rows = int(np.ceil(n_crypto / n_cols))

plt.figure(figsize=(14, 3 * n_rows))

for i, crypto in enumerate(crypto_names):
    plt.subplot(n_rows, n_cols, i + 1)
    plt.plot(future_dates, predictions[:, i], marker="o",  linewidth=2, label=crypto, color="tab:red")
    plt.title(f"Prediksi {crypto} 30 Hari ke Depan (Mulai 25 Juli 2025)")
    plt.xlabel("Tanggal")
    plt.ylabel("Harga (Normalized)")
    plt.xticks(rotation=45)
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.legend()

plt.tight_layout()
plt.show()